# Construyendo un sistema de análisis de sentimiento (III)

## Tarea 7b

## Enunciado

Uso de Sentiwordnet para mejorar el sistema previo

Incluya en el sistema anterior los atributos que considere oportunos vistos en la sesión de hoy basado en el uso de la herramienta Sentiwordnet y aplique aprendizaje. ¿Se mejoran los resultados?


## Implementación

### Paso 1: Descargando las librerías de nltk

Importamos la librería nltk. Si no la tenemos instalada la podemos instalar mediante pip

    pip install nltk
 
Una vez importada, descargamos los paquetes necesarios: en este caso, con los conjuntos "popular" y "spanish grammars" será suficiente. El "popular" descargará los paquetes mas populares, mientras que "spanish grammars" descargará los paquetes para realizar el procesamiento de texto en español

In [1]:
import nltk


nltk.download('popular')
nltk.download('spanish_grammars')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk

True

### Paso 2: Cargando el corpus

Cargamos el corpus de entrenamiento desde el fichero "train_es.tsv", y el corpus de prueba desde el fichero "trial_es.tsv". Este fichero utiliza un formato csv separado por tabulaciones. Como columnas cargamos "text", correspondiente al texto del tuit; y "HS", correspondiente a las etiquetas binarias de odio/no_odio

Para ello utilizaremos la librería pandas, con el método `read_csv()`, indicando en sus parámetros el tipo de separador y el número de filas a leer. Esto nos cargará las 10 filas en un dataframe (al que llamaremos `corpus_df`) con las mismas columnas que en el fichero original, etiquetadas por sus respectivos nombres. 

Una vez cargado, mostraremos el dataframe para comprobar que se ha cargado correctamente

In [2]:
import pandas as pd

corpus_df_train = pd.read_csv("HateEval/train_es.tsv", sep="\t", usecols =["text","HS"])
labels_train = corpus_df_train[['HS']]
print(labels_train)


corpus_df_eval = pd.read_csv("HateEval/trial_es.tsv", sep="\t", usecols =["text", "HS"])
labels_eval = pd.read_csv("HateEval/trial_es.tsv", delimiter='\t', usecols=["HS"])

print(corpus_df_train)
print(corpus_df_eval)

      HS
0      1
1      1
2      0
3      0
4      0
...   ..
4464   1
4465   1
4466   0
4467   1
4468   0

[4469 rows x 1 columns]
                                                   text  HS
0     Easyjet quiere duplicar el número de mujeres p...   1
1     El gobierno debe crear un control estricto de ...   1
2     Yo veo a mujeres destruidas por acoso laboral ...   0
3     — Yo soy respetuoso con los demás, sólamente l...   0
4     Antonio Caballero y como ser de mal gusto e ig...   0
...                                                 ...  ..
4464  @miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...   1
4465  @IvanDuque presidente en Cúcuta , tenemos prob...   1
4466              - Callaté Visto Que Te Dejo En Puta🎤🎶   0
4467  -¿porque los hombres se casan con las mujeres?...   1
4468  — No hay nada más lento que un caracol. — Cáll...   0

[4469 rows x 2 columns]
                                                 text  HS
0   @ian_delaCalva @IrantzuVarela @pikaramagazine ...   0
1   NI

### Paso 3: Separando el texto en tokens y clasificando

En este paso, realizamos la clasificación de las palabras en función de su categoría gramatical.

Antes de realizar la clasificación, separamos el texto de cada tuit en tokens. Para ello, utilizamos la clase `TweetTokenizer`, especializada en la extracción de tokens de tuits. Con esto, transformaremos en campo `'text'` de cada tuit en una lista de tokens, lo cual nos facilitará dicha clasificación

Tras esto, pasaremos cada token a minúsculas, eliminaremos stopwords y tokens no alfanuméricos (hashtag, citas, enlaces...) y los añadiremos a una lista de palabras.


Finalmente, aplicaremos el POS Tagger, que generará una columna añadiendo la categoría gramatical de cada palabra

In [3]:
from nltk.tokenize import TweetTokenizer
from nltk.tag.stanford import StanfordPOSTagger
from nltk.corpus import stopwords

stopwords.words('spanish')

spanish_postagger = StanfordPOSTagger('./StanfordTagger/spanish.tagger', './StanfordTagger/stanford-postagger.jar', encoding='utf8')
cleantokens = []

tweet_tokenizer = TweetTokenizer()

for index, tweet in corpus_df_eval.iterrows():
    for word in tweet_tokenizer.tokenize(tweet['text']):
        word = word.lower()
        if word.isalnum() and word not in stopwords.words('spanish'):
            cleantokens.append(word)

## Ejercicio 2: Caracterización gramatical

### Paso 1: Obtener tokens del dataframe

De nuevo, tokenizamos el campo "text" del dataframe, transformando dicha columna en una lista de palabras de cada tuit. Repetimos el mismo proceso que en el ejercicio anterior, aplicando el `TweetTokenizer()`, pasando palabras a minúsculas, y eliminando stopwords y palabras con caracteres no alfanuméricos

Aplicamos el proceso con ambos conjuntos: de entrenamiento y de prueba

In [4]:
def token_split(wordlist: pd.DataFrame):
    token_list = tweet_tokenizer.tokenize(wordlist)
    
    words_no_stop = [word.lower() for word in token_list if word not in stopwords.words('spanish') and word.isalnum()]
    return words_no_stop

corpus_filtered_train = corpus_df_train.copy()
corpus_filtered_train['text'] = corpus_df_train['text'].apply(token_split)

corpus_filtered_eval = corpus_df_eval.copy()
corpus_filtered_eval['text'] = corpus_df_eval['text'].apply(token_split)

print(corpus_filtered_train)
print(corpus_filtered_eval)

                                                   text  HS
0     [easyjet, quiere, duplicar, número, mujeres, p...   1
1     [el, gobierno, debe, crear, control, estricto,...   1
2     [yo, veo, mujeres, destruidas, acoso, laboral,...   0
3     [yo, respetuoso, demás, sólamente, recuerdo, y...   0
4     [antonio, caballero, ser, mal, gusto, ignorant...   0
...                                                 ...  ..
4464                          [cállateeee, zorra, ahre]   1
4465  [presidente, cúcuta, problemas, venezolanos, d...   1
4466          [callaté, visto, que, te, dejo, en, puta]   0
4467  [hombres, casan, mujeres, cabras, saben, frega...   1
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0

[4469 rows x 2 columns]
                                                 text  HS
0                        [oye, molestas, puta, madre]   0
1                          [ninguna, mujer, es, puta]   0
2   [editar, además, complicado, hace, merezca, pe...   0
3   [bien, joder, puta,

## Ejercicio 3: Filtrado gramatical

### Filtrado de categorías gramaticales

En este ejercicio, debemos filtrar varias categorías gramaticales del texto.
Para ello, nos vamos a crear la función `filter_categories` que permitirá filtrar las palabras que se correspondan a las categorías gramaticales indicadas por parámetro. Estas se indican mediante su código ("a" = Adjetivos, "n" = Nombres... etc)

Filtramos las palabras que se correspondan a nombres, adjetivos o verbos. 

Creamos una nueva columna llamada "pos_tags" en la que asociaremos cada palabra con su categoría gramatical.
Las palabras que no se hayan podido etiquetas las consideraremos como si fueran nombres.

Aplicamos el POSTagger sobre nuestros corpus de entrenamiento y evaluación. Creamos un diccionario para asociar las etiquetas de nuestro tagger a las de WordNet.


In [36]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tag.stanford import StanfordPOSTagger

spanish_postagger = StanfordPOSTagger('./StanfordTagger/spanish.tagger', './StanfordTagger/stanford-postagger.jar', encoding='utf8')

penn_to_wn = {"a": wn.ADJ, "r": wn.ADV, "n": wn.NOUN, "v": wn.VERB}
#penn_to_wn = {"ADJ": wn.ADJ, "ADV": wn.ADV, "NOUN": wn.NOUN, "VERB": wn.VERB, None: wn.NOUN}

def filter_categories(row: pd.DataFrame, category_list: list):
    tagged_words = spanish_postagger.tag(row['text'])    
    
    tokens_filtered = []
    postags = []
    
    for elem in tagged_words:
        if elem[1][0] in category_list:
            tokens_filtered.append(elem[0])
            postags.append((elem[0], penn_to_wn[elem[1][0]]))
    
    row['text'] = tokens_filtered
    
    # Replace Penn tags with wn equivalents
    row['pos_tags'] = postags
        
    return row

corpus_gramfiltered_train = corpus_filtered_train.copy().apply(filter_categories, args=[penn_to_wn.keys()], axis=1)
corpus_gramfiltered_eval = corpus_filtered_eval.copy().apply(filter_categories, args=[penn_to_wn.keys()], axis=1)
print(corpus_gramfiltered_eval)
print(corpus_gramfiltered_train)

                                                 text  HS  \
0                             [molestas, puta, madre]   0   
1                                   [mujer, es, puta]   0   
2   [editar, además, complicado, hace, merezca, pe...   0   
3            [joder, puta, alegría, mereces, pequeña]   0   
4   [política, levanta, sesión, hijos, puta, manda...   0   
..                                                ...  ..   
95  [imagen, pokemon, bailando, machupichu, hoy, c...   0   
96  [ciudadano, ruso, denis, yakovlev, facilitó, f...   0   
97  [toy, feliz, voy, comer, empanadas, árabes, vi...   0   
98  [barrios, pueblo, solidario, caos, acogida, in...   0   
99  [viernes, noche, moritos, reggaeton, arabe, de...   0   

                                             pos_tags  
0              [(molestas, a), (puta, n), (madre, n)]  
1                    [(mujer, n), (es, v), (puta, a)]  
2   [(editar, v), (además, r), (complicado, a), (h...  
3   [(joder, v), (puta, n), (alegría, n), (

## Clasificación de sentimientos

### Obtención de sentimientos

Creamos la función `get_sentiment()` que permite obtener los sentimientos asociados a cada palabra.
Para ello, obtenemos su lema, su synset y su senti_synset, y generamos una tupla con el nombre del synset, la puntuación del sentisynset positivo y negativo, y la puntuación global.

Para obtener el synset, puesto que WordNet per se no soporta el idioma español, añadimos la extensión OpenMultiWordnet (omw)

In [186]:
nltk.download('omw')

def get_sentiment(word,tag):             
    #Synset es un tipo especial de interfaz simple que está presente en NLTK para buscar palabras en WordNet.
    #Las instancias #Synset son agrupaciones de palabras sinónimos que expresan el mismo concepto.
    #Algunas de las palabras tienen solo un Synset y otras tienen varios.
    synsets = wn.synsets(word, pos=tag, lang="spa")
    
    if not synsets:
        return []
    
    # Tomamos la primera acepción, la más común
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name()) #y la buscamos en sentiwordnet

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

[nltk_data] Downloading package omw to /home/almu/nltk_data...
[nltk_data]   Package omw is already up-to-date!


### Ponderación de sentimientos

Utilizando la función anterior, realizamos la ponderación del sentimiento global de cada frase.
Obtenemos la ponderación positiva y negativa sumando las de cada palabra de la frase, y calculamos el valor final restando la puntuación negativa a la positiva.

Los resultados los almacenamos en una nueva columna llamada `senti_score`

In [187]:
senti_score = []

def calculate_sentiment(row: pd.DataFrame):
    pos=neg=0
    pos_val = row['pos_tags']
    
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    
    for score in senti_val:
        try:
            pos = pos + score[1]  #la puntuación positiva se almacena en la segunda posición
            neg = neg + score[2]  #la puntuación negativa se almacena en la tercera posición
        except:
            continue
    
    row['senti_score'] = pos - neg
                
    return row
    
corpus_sentiscore_train = corpus_gramfiltered_train.copy().apply(calculate_sentiment, axis=1)
print(corpus_sentiscore_train['senti_score'])

print(corpus_sentiscore_train.head)

corpus_sentiscore_eval = corpus_gramfiltered_eval.copy().apply(calculate_sentiment, axis=1)
print(corpus_sentiscore_eval['senti_score'])

print(corpus_sentiscore_eval)


0      -0.375
1      -0.500
2      -1.875
3       0.500
4       0.500
        ...  
4464    0.000
4465    0.000
4466    0.000
4467    0.250
4468    0.000
Name: senti_score, Length: 4469, dtype: float64
<bound method NDFrame.head of                                                    text  HS  \
0     [easyjet, quiere, duplicar, número, mujeres, p...   1   
1     [gobierno, debe, crear, control, estricto, inm...   1   
2     [veo, mujeres, destruidas, acoso, laboral, cal...   0   
3     [respetuoso, sólamente, recuerdo, escoria, cul...   0   
4     [antonio, caballero, ser, mal, gusto, ignorant...   0   
...                                                 ...  ..   
4464                          [cállateeee, zorra, ahre]   1   
4465  [presidente, cúcuta, problemas, venezolanos, d...   1   
4466                       [callaté, visto, dejo, puta]   0   
4467  [hombres, casan, mujeres, cabras, saben, frega...   1   
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0   

           

### Etiquetado de sentimientos

Una vez con las ponderaciones ya obtenidas, etiquetamos los sentimientos según un rango.
Aquellos cuyo valor sea superior a 0.05 se considerarán positivos, los menores a -0.05 serán negativos, y el resto serán neutrales.

Almacenamos estas etiquetas en una nueva columna llamada `Overall_Sentiment`

In [188]:
overall=[]

def tag_sentiments(data: pd.DataFrame):
    if data['senti_score']>= 0.05:
        overall = 'Positive'
    elif data['senti_score']<= -0.05:
        overall = 'Negative'
    else:
        overall= 'Neutral'
        
    data['Overall_Sentiment'] = overall

    return data
    
corpus_tagsents_train = corpus_sentiscore_train.copy().apply(tag_sentiments, axis=1)
print(corpus_tagsents_train.head)

corpus_tagsents_eval = corpus_sentiscore_eval.copy().apply(tag_sentiments, axis=1)
print(corpus_tagsents_eval.head)

<bound method NDFrame.head of                                                    text  HS  \
0     [easyjet, quiere, duplicar, número, mujeres, p...   1   
1     [gobierno, debe, crear, control, estricto, inm...   1   
2     [veo, mujeres, destruidas, acoso, laboral, cal...   0   
3     [respetuoso, sólamente, recuerdo, escoria, cul...   0   
4     [antonio, caballero, ser, mal, gusto, ignorant...   0   
...                                                 ...  ..   
4464                          [cállateeee, zorra, ahre]   1   
4465  [presidente, cúcuta, problemas, venezolanos, d...   1   
4466                       [callaté, visto, dejo, puta]   0   
4467  [hombres, casan, mujeres, cabras, saben, frega...   1   
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0   

                                               pos_tags  senti_score  \
0     [(easyjet, n), (quiere, v), (duplicar, v), (nú...       -0.375   
1     [(gobierno, n), (debe, v), (crear, v), (contro...       -0.500 

### Añadiendo etiquetas

Añadimos las nuevas columnas como etiquetas del corpus, que utilizaremos posteriormente en la fase de entrenamiento

In [180]:
#labels_eval['senti_score'] = corpus_tagsents_eval['senti_score'].copy()
labels_eval = corpus_tagsents_eval['Overall_Sentiment'].copy()

#labels_train['senti_score'] = corpus_tagsents_train['senti_score'].copy()
labels_train = corpus_tagsents_train['Overall_Sentiment'].copy()

## Preparando la bolsa de palabras
### Generando bolsa de nombres, adjetivos y adverbios

In [189]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

corpus_tagsents_str_train = corpus_tagsents_train.copy()
corpus_tagsents_str_eval = corpus_tagsents_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_tagsents_str_train["text"] = corpus_tagsents_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_tagsents_str_eval["text"] = corpus_tagsents_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
tagsents_vectorizer_train = TfidfVectorizer()
tagsents_vectorizer_eval = TfidfVectorizer()

# construimos vocabulario
vector_tagsents_train = tagsents_vectorizer_train.fit_transform(corpus_tagsents_str_train["text"])
vector_tagsents_eval = tagsents_vectorizer_eval.fit_transform(corpus_tagsents_str_eval["text"])

print(tagsents_vectorizer_train.vocabulary_)
print(tagsents_vectorizer_eval.vocabulary_)

{'easyjet': 4101, 'quiere': 10254, 'duplicar': 4062, 'número': 8539, 'mujeres': 8222, 'piloto': 9467, 'verás': 12605, 'aparcar': 773, 'avión': 1198, 'gobierno': 5522, 'debe': 3306, 'crear': 3010, 'control': 2867, 'estricto': 4727, 'inmigración': 6417, 'zonas': 13019, 'fronterizas': 5285, 'colombia': 2526, 'después': 3697, 'querrán': 10239, 'venir': 12553, 'masa': 7644, 'veo': 12564, 'destruidas': 3714, 'acoso': 193, 'laboral': 6904, 'callejero': 1826, 'depresión': 3541, 'debido': 3321, 'violación': 12697, 'sexual': 11277, 'maltrato': 7473, 'físico': 5356, 'conocí': 2766, 'suicidaron': 11649, 'tipo': 11977, 'comportamientos': 2648, 'machistas': 7369, 'vas': 12474, 'seguir': 11169, 'show': 11305, 'pobre': 9587, 'respetuoso': 10678, 'sólamente': 11714, 'recuerdo': 10459, 'escoria': 4550, 'culpa': 3165, 'claro': 2412, 'sé': 11711, 'tomas': 12042, 'antonio': 751, 'caballero': 1692, 'ser': 11240, 'mal': 7432, 'gusto': 5708, 'ignorante': 6160, 'vez': 12621, 'conductas': 2719, 'componen': 2645

## Entrenando los modelos

### Paso 1: Dividiendo las colecciones en subconjuntos

#### Generando subconjuntos para bolsa de nombres, adjetivos y adverbios

In [190]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_tagsents_train, X_tagsents_test, y_tagsents_train, y_tagsents_test = train_test_split(vector_tagsents_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_tagsents_train)
print(y_tagsents_train)
print(labels_train)

  (0, 6480)	0.3361598341595549
  (0, 10835)	0.3361598341595549
  (0, 5875)	0.3361598341595549
  (0, 4061)	0.3361598341595549
  (0, 7026)	0.3361598341595549
  (0, 11249)	0.27405829620596683
  (0, 326)	0.3205146162822447
  (0, 5068)	0.27405829620596683
  (0, 9805)	0.30941415754263946
  (0, 2470)	0.2937689396653292
  (1, 12004)	0.22449358258188853
  (1, 8857)	0.22449358258188853
  (1, 4032)	0.22449358258188853
  (1, 12524)	0.19798554182039432
  (1, 3379)	0.23545174434153415
  (1, 10163)	0.4031887631456792
  (1, 1733)	0.23545174434153415
  (1, 11260)	0.23545174434153415
  (1, 10192)	0.21671864308096428
  (1, 7088)	0.22449358258188853
  (1, 7940)	0.2057604813213186
  (1, 8956)	0.15888088903757472
  (1, 4905)	0.1848641336535322
  (1, 4624)	0.15659562819156017
  (1, 13080)	0.1651110565414574
  :	:
  (3571, 426)	0.2903588235886193
  (3571, 12429)	0.4556775329273449
  (3571, 6418)	0.30778759552289553
  (3572, 9171)	0.4423273284606091
  (3572, 7092)	0.4217409682722883
  (3572, 12295)	0.421740968

### Paso 2: Ajustando los modelos del clasificador

Utilizaremos el clasificador SVC, debido a que el MLP es incapaz de converger

#### Ajustando el modelo para la bolsa de nombres, adjetivos y adverbios

In [191]:
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report

clasificador_tagsents_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_tagsents_svc.fit(X_tagsents_train, np.ravel(y_tagsents_train))
pred_y_tagsents_svc = clasificador_tagsents_svc.predict(X_tagsents_test)

print("CCR: %f"%(clasificador_tagsents_svc.score(X_tagsents_test, y_tagsents_test)))
print(classification_report(y_tagsents_test, pred_y_tagsents_svc))

CCR: 0.438479
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       261
     Neutral       0.44      1.00      0.61       392
    Positive       0.00      0.00      0.00       241

    accuracy                           0.44       894
   macro avg       0.15      0.33      0.20       894
weighted avg       0.19      0.44      0.27       894



/home/almu/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/almu/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/almu/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Probando los modelos en datos reales

Una vez convergidos los modelos, los aplicamos sobre datos reales, utilizando para ello el conjunto de evaluación (eval)

### Prueba con nombres, adjetivos y adverbios

In [192]:
eval_tagsents = tagsents_vectorizer_train.transform(corpus_tagsents_str_eval['text'])
evalPredict_tagsents_svc = clasificador_tagsents_svc.predict(eval_tagsents)

print("CCR: %f"%(clasificador_tagsents_svc.score(eval_tagsents, labels_eval)))
print(classification_report(labels_eval, evalPredict_tagsents_svc))
print(evalPredict_tagsents_svc)

CCR: 0.390000
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        28
     Neutral       0.39      1.00      0.56        39
    Positive       0.00      0.00      0.00        33

    accuracy                           0.39       100
   macro avg       0.13      0.33      0.19       100
weighted avg       0.15      0.39      0.22       100

['Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral' 'Neutral'
 'Neutral' 'Neutral' 'Neutral' 'Neutr

/home/almu/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/almu/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/almu/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Conclusiones

El TweetTokenizer es una gran herramienta para extraer tokens de mensajes provenientes de Twitter. Mejora la identificación de tokens con características especiales, como hashtags, menciones, URLs... Esto permite realizar un filtrado posterior de forma mas precisa, sin necesidad de recurrir a expresiones regulares.

Respecto al POS Tagging, hemos descubierto que es una herramienta bastante poderosa para realizar un análisis gramatical. Por contra, la clasificación gramatical no es perfecta, y el tiempo de cómputo es bastante alto.

Tras realizar el filtrado de nombres, adjetivos, verbos y adverbios, hemos aplicado un análisis de sentimientos no supervisado, generando una clasificación entre sentimientos negativos, positivos y neutros. La columna 'Overall_Sentiment' resultante la hemos utilizado como etiqueta para el sistema de clasificación. 

Se aprecia como hay un gran desequilibrio en las clases, siendo la mayoría de elementos de la clase Neutral, incluso habiendo aplicado la estratificación en la división del conjunto. Esto provoca que el modelo quede sobreajustado a dicha clase, provocando que la precisión en las otras dos sea casi nula.

Como posibles causas de este problema, se sugiere el soporte limitado de WordNet y SentiWordNet para el idioma español, que provoca que una gran parte de las palabras no sean reconocidas por estos.